# Knapsack problem

In [8]:
%load_ext autoreload
%autoreload 2

In [71]:
from src import qubo, data, ising, brute_force, cplex
import numpy as np
from time import time

## Select the problem

In [2]:
kp_type = 'small'
problems = data.load(kp_type, path='kp_instances')
problem = problems[0]
problem

{'C': 10,
 'n': 8,
 'weights': [6, 6, 7, 4, 3, 3, 2, 2],
 'profits': [6, 7, 7, 4, 3, 4, 2, 1]}

# Classical solvers

## Brute force

In [90]:
bf_sol = None
if kp_type == 'large':
    print('I CANNOT DO THAT SIR!')
else:
    max_ram = 40E9
    st = time()
    bf_sol = brute_force.kp_brute_force(problem['profits'], problem['weights'], problem['C'], max_ram)
    et = time()
    bf_time = et - st
bf_sol

Number of threads: 15
Max samples: 31250000
Number of steps: 1


100%|██████████| 1/1 [00:00<00:00, 55.21it/s]


{'profit': array([11, 11, 11, 11]),
 'cost': array([10, 10,  9, 10]),
 'combo': [array([3, 4, 5]), array([2, 5]), array([1, 5]), array([1, 3])]}

In [91]:
# get the solution which also minimises the cost
best_indices = np.flatnonzero(bf_sol['cost'] == bf_sol['cost'].min())

print('Best solution(s) found by brute force:')
print('Profit:', bf_sol['profit'][best_indices[0]])
print('Cost:', bf_sol['cost'][best_indices[0]])
print('Combo(s):', [bf_sol['combo'][i] for i in best_indices])

print('============')
print('  RUN INFO  ')
print('============')
print(f'Run time: {bf_time :.4f} s')


Best solution(s) found by brute force:
Profit: 11
Cost: 9
Combo(s): [array([1, 5])]
  RUN INFO  
Run time: 0.0217 s


## CPlex

In [95]:
cplex_sol = cplex.cplex_kp_solver(problem['profits'], problem['weights'], problem['C'], problem['n'])
print('Solution found by CPLEX:')
print('Profit:', cplex_sol['profit'])
print('Cost:', cplex_sol['cost'])
print('Combo:', np.flatnonzero(cplex_sol['combo']))

print('===========================')
print('         RUN INFO          ')
print('===========================')
cplex_time = cplex_sol['runtime']
print(f'Run time: {cplex_time:.4f} s')

Knapsack problem instance as a BLIP
Problem name: KP_0008_00010

Maximize
  6*x_0 + 7*x_1 + 7*x_2 + 4*x_3 + 3*x_4 + 4*x_5 + 2*x_6 + x_7

Subject to
  Linear constraints (1)
    6*x_0 + 6*x_1 + 7*x_2 + 4*x_3 + 3*x_4 + 3*x_5 + 2*x_6 + 2*x_7
    <= 10  'The total weight of the knapsack must not exceed 10'

  Binary variables (8)
    x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7




Solution found by CPLEX:
Profit: 11.0
Cost: 9
Combo: [1 5]
         RUN INFO          
Run time: 0.0150 s


# Quantum solvers

## Exact diagonalization

In [92]:
# get the qubo matrix
LAMBDA = max(problem['profits']) + 2
st = time()
Q = qubo.get_Q(problem['weights'], problem['profits'], problem['C'], LAMBDA)
t1 = time() - st

# embed the qubo matrix into an hamiltonian
st = time()
h, J, offset = ising.qubo_to_hamiltonian(Q)
t2 = time() - st

if kp_type != 'small':
    print('I CANNOT DO THAT SIR!')
else:
    # get the diagonal of the hamiltonian
    st = time()
    diag = ising.construct_quantum_hamiltonian_scipy(h, J, offset)
    t3 = time() - st
    st = time()
    diag_sol = np.argmin(diag)
    diag_sol_bits = (~brute_force.toBitstrings(diag_sol, Q.shape[0])).astype(int)
    t4 = time() - st
    cost = diag_sol_bits[:problem['n']].dot(problem['weights'])
    profit = diag_sol_bits[:problem['n']].dot(problem['profits'])
    ed_time = t1 + t2 + t3 + t4
    print('Solution found by diagonalising the Hamiltonian:')
    print('Profit:', profit)
    print('Cost:', cost, f" (but cost was: {problem['C']}" if cost > problem['C'] else '')
    print('Combo:', np.flatnonzero(diag_sol_bits[:problem['n']]))
    print('============')
    print('  RUN INFO  ')
    print('============')
    print(f'Time for QUBO: {t1:.4f} s')
    print(f'Time for Hamiltonian coeffs: {t2:.4f} s')
    print(f'Time for Hamiltonian: {t3:.4f} s')
    print(f'Time for diagonalisation: {t4:.4f} s')
    print(f'Total time: {ed_time:.4f} s')

Solution found by diagonalising the Hamiltonian:
Profit: 10
Cost: 10 
Combo: [4 5 6 7]
  RUN INFO  
Time for QUBO: 0.0002 s
Time for Hamiltonian coeffs: 0.0004 s
Time for Hamiltonian: 0.0302 s
Time for diagonalisation: 0.0001 s
Total time: 0.0310 s


## QAOA in Qiskit

In [57]:
from qiskit.result import QuasiDistribution

from qiskit_algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA

from qiskit_algorithms.utils import algorithm_globals
from qiskit.circuit.library import QAOAAnsatz
from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector

def calc_cost_and_constraint(profits, weights, solution, C):
    assert len(profits) == len(weights)
    s = len(profits)

    total_cost = sum(profits[i] * solution[i] for i in range(s))
    print("Optimum value of profit function from ED = ",total_cost)

    slack = sum((2**i) * solution[s + i] for i in range(len(solution) - (s + 1)))
    slack += (C + 1 - 2**(len(solution) - s - 1)) * solution[-1]
    print("Total value of slack variable = ", slack)

    total_weight = sum(weights[k] * solution[k] for k in range(s))
    if (total_weight <= C):
       print("Total weight less than given constraint")
    else:
       print("Weight constraint not satisfied; wrong solution")

In [31]:
# get the qubo matrix
LAMBDA = max(problem['profits']) + 2
Q = qubo.get_Q(problem['weights'], problem['profits'], problem['C'], LAMBDA)

# embed the qubo matrix into an hamiltonian
h, J, offset = ising.qubo_to_hamiltonian(Q)

st = time()
H = ising.construct_quantum_hamiltonian_qiskit(h, J, offset)
t1 = time() - st

In [60]:
sampler = Sampler()
algorithm_globals.random_seed = 10598

optimizer = COBYLA()
qaoa = QAOA(sampler, optimizer, reps=2)

result = qaoa.compute_minimum_eigenvalue(H)

In [96]:
# get the solution
qaoa_qk_sol = result.best_measurement['bitstring']
qaoa_qk_sol_bits = np.logical_not(np.array([int(bit) for bit in qaoa_qk_sol])).astype(int)
print('Solution found by QAOA:')
print('Profit:', qaoa_qk_sol_bits[:problem['n']].dot(problem['profits']))
print('Cost:', qaoa_qk_sol_bits[:problem['n']].dot(problem['weights']))
print('Combo:', np.flatnonzero(qaoa_qk_sol_bits[:problem['n']]))
print('Slack:', qaoa_qk_sol_bits[problem['n']:])
calc_cost_and_constraint(problem['profits'], problem['weights'], qaoa_qk_sol_bits, problem['C'])
print('============')
print('  RUN INFO  ')
print('============')
qubo_qk_time = result.optimizer_time + t1
print(f'Time for Hamiltonian: {t1:.4f} s')
print(f'Time for QAOA: {result.optimizer_time:.4f} s')
print(f'Total time: {qubo_qk_time:.4f} s')

Solution found by QAOA:
Profit: 10
Cost: 10
Combo: [4 5 6 7]
Slack: [0 0 0 0]
Optimum value of profit function from ED =  10
Total value of slack variable =  0
Total weight less than given constraint
  RUN INFO  
Time for Hamiltonian: 0.0002 s
Time for QAOA: 11.0921 s
Total time: 11.0924 s


## QAOA in Matcha Tea